In [1]:
import numpy as np
import edward as ed
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
def build_toy_dataset(N):
    theta = [[0.1, 0.2, 0.7], [0.7,0.2,0.1], 
                      [0.2,0.7,0.1], [0.7,0.1,0.2]] # 4 Documents, 3 Topics
    phi = [[0.1,0.1,0.1,0.1,0.1,0.5], 
                    [0.1,0.1,0.1,0.1,0.5,0.1], 
                    [0.2,0.2,0.2,0.2,0.1,0.1], # 6 Words per topic
                   ]
    x = np.zeros((N*4, 2), dtype=np.int32)
    x[:,1] = [item for item,count in [(0,N), (1,N),(2,N),(3,N)] for i in range(count)]
    for n in range(x.shape[0]):
        k = np.argmax(np.random.multinomial(1, theta[x[n,1]])) # which topic depending on document
        x[n, 0] = np.argmax(np.random.multinomial(1, phi[k]))

    return x

D = 4  # number of documents
M = 100  # number of words per document

x_train = build_toy_dataset(M)

In [3]:
K = 3 # Number of topics
N = x_train.shape[0]
V = len(set(x_train[:,0])) # Unique words observed

In [4]:
x_train[95:104] # Word ID| Document ID

array([[4, 0],
       [2, 0],
       [1, 0],
       [4, 0],
       [4, 0],
       [0, 1],
       [4, 1],
       [3, 1],
       [3, 1]], dtype=int32)

In [5]:
x_train.shape

(400, 2)

In [6]:
from edward.models import Dirichlet, Categorical, Empirical

In [7]:
doc   = tf.placeholder(tf.int32, N)

theta = Dirichlet(tf.zeros([D,K]) + 1/K)
phi   = Dirichlet(tf.zeros([K,V]) + 1/V)

z   = Categorical(tf.gather(theta, tf.gather(doc, tf.range(N))))
w   = Categorical(tf.gather(phi,   tf.gather(z,   tf.range(N))))

In [8]:
qz     = Empirical(params=tf.Variable(tf.zeros([1, N], dtype=tf.int32)))
qw     = Empirical(params=tf.Variable(tf.zeros([1, N], dtype=tf.int32)))
qtheta = Empirical(params=tf.Variable(tf.zeros([1, D, K], dtype=tf.float32) + 1/K))
qphi   = Empirical(params=tf.Variable(tf.zeros([1, K, V], dtype=tf.float32) + 1/V))

In [8]:
with tf.Session() as sess:
    print(ed.inferences.complete_conditional(z))

AttributeError: 'NoneType' object has no attribute 'shape'

In [22]:
from edward.inferences import Gibbs

In [25]:
its = 500
inference = ed.SGLD({z: qz, w: qw, theta: qtheta, phi: qphi}, 
                    data={w: x_train[:,0], doc: x_train[:,1]})
inference.run(n_iter=its) #, n_print=100, n_samples=10)

/Users/hauptjoh/anaconda/envs/edward/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/Users/hauptjoh/anaconda/envs/edward/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


ValueError: 'transform' does not handle supports of type 'categorical'